# CS2 Playoff Winner Prediction (Version 2)

This notebook predicts CS2 playoff match winners and the tournament champion for the BLAST Austin Major using XGBoost. Enhancements include weighted rolling stats, team tier features, recent match filtering, BO3 simulation, and improved calibration to align with betting odds.

In [5]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
csvs = ['hltv_match_stats_0_3000_1506.csv', 'hltv_match_stats_3100_6000_1506.csv']
# csvs = ['hltv_match_stats_2106.csv', 'hltv_match_stats_0_3000_1506.csv', 'hltv_match_stats_3100_6000_1506.csv']
df = pd.concat([pd.read_csv(f) for f in csvs])

# Filter recent matches (approximate last 6 months by highest GameIDs)
df['GameID'] = df['GameID'].astype(int)
recent_threshold = df['GameID'].quantile(0.5)  # Top 50% of GameIDs
df = df[df['GameID'] >= recent_threshold]

# Display first few rows
df.head()

,Team,Map,Map Won,Picks,Bans,Player,Nationality,Kills,Assists,Deaths,...,FK Diff,Rating,Team Total Score,Team CT Half,Team T Half,Team Rating,First Kills,Clutches Won,Match URL,GameID
0,Natus Vincere,Mirage,Yes,NaN,NaN,jL,Lithuania,30,5,14,...,2,1.96,13,7.0,6.0,1.12,10,4,https://www.hltv.org/matches/2382609/natus-vin...,201539
1,Natus Vincere,Mirage,Yes,NaN,NaN,iM,Romania,20,6,14,...,0,1.37,13,7.0,6.0,1.12,10,4,https://www.hltv.org/matches/2382609/natus-vin...,201539
2,Natus Vincere,Mirage,Yes,NaN,NaN,w0nderful,Ukraine,14,3,14,...,2,0.87,13,7.0,6.0,1.12,10,4,https://www.hltv.org/matches/2382609/natus-vin...,201539
3,Natus Vincere,Mirage,Yes,NaN,NaN,b1t,Ukraine,10,8,18,...,-1,0.77,13,7.0,6.0,1.12,10,4,https://www.hltv.org/matches/2382609/natus-vin...,201539
4,Natus Vincere,Mirage,Yes,NaN,NaN,Aleksib,Finland,9,8,19,...,-6,0.61,13,7.0,6.0,1.12,10,4,https://www.hltv.org/matches/2382609/natus-vin...,201539


## Data Preprocessing

- Remove duplicates and missing data
- Convert percentage strings to floats
- Compute exponentially weighted rolling stats
- Add team tier, map-specific, and head-to-head features
- Cap outliers in differential features

In [6]:
# Remove duplicates and matches with missing critical stats
df = df.drop_duplicates(subset=['GameID', 'Team', 'Player'])
df = df.dropna(subset=['Team Rating', 'Kills', 'KAST', 'ADR', 'First Kills'])

# Convert KAST to float
df['KAST'] = df['KAST'].str.rstrip('%').astype(float) / 100

# Handle missing values
df.fillna({'Picks': 'None', 'Bans': 'None'}, inplace=True)

# Aggregate player stats to team-level
team_stats = df.groupby(['GameID', 'Team', 'Map', 'Map Won', 'Team Total Score', 'Team CT Half', 'Team T Half', 'Team Rating', 'First Kills', 'Clutches Won']).agg({
    'Kills': 'mean',
    'Assists': 'mean',
    'Deaths': 'mean',
    'KAST': 'mean',
    'K-D Diff': 'mean',
    'ADR': 'mean',
    'FK Diff': 'mean',
    'Rating': 'mean'
}).reset_index()

# Compute team tier based on average Team Rating
team_tiers = team_stats.groupby('Team')['Team Rating'].mean().reset_index(name='Team_Tier')
team_tiers['Team_Tier'] = pd.qcut(team_tiers['Team_Tier'], q=3, labels=[1, 2, 3])  # 1=low, 2=mid, 3=high

# Get numeric columns for rolling statistics
numeric_cols = ['Team Rating', 'Kills', 'Assists', 'Deaths', 'KAST', 'K-D Diff', 'ADR', 'FK Diff', 'Rating', 'Team Total Score', 'First Kills']

# Compute exponentially weighted rolling stats (span=10 matches)
team_stats = team_stats.sort_values('GameID')
rolling_stats = pd.DataFrame()

# Calculate ewm for each numeric column separately
for col in numeric_cols:
    ewm_result = team_stats.groupby('Team')[col].ewm(span=10, min_periods=1).mean().reset_index()
    if rolling_stats.empty:
        rolling_stats = ewm_result
    else:
        rolling_stats = rolling_stats.merge(ewm_result, on=['Team', 'level_1'])

# Rename columns
rolling_stats = rolling_stats.rename(columns={
    'level_1': 'GameID',
    'Team Rating': 'Rolling_Rating',
    'Kills': 'Rolling_Kills',
    'KAST': 'Rolling_KAST',
    'ADR': 'Rolling_ADR',
    'Team Total Score': 'Rolling_Score',    
    'First Kills': 'Rolling_First_Kills'
})

# Normalize rolling stats
scaler_stats = StandardScaler()
rolling_cols = ['Rolling_Rating', 'Rolling_Kills', 'Rolling_KAST', 'Rolling_ADR', 'Rolling_Score', 'Rolling_First_Kills']
rolling_stats[rolling_cols] = scaler_stats.fit_transform(rolling_stats[rolling_cols])

# Compute map-specific win rates
map_wins = team_stats.groupby(['Team', 'Map'])['Map Won'].apply(lambda x: (x == 'Yes').mean()).reset_index(name='Map_Win_Rate')

# Compute head-to-head win rates
h2h_matches = team_stats.groupby('GameID').apply(lambda x: pd.Series({
    'Team_A': x.iloc[0]['Team'],
    'Team_B': x.iloc[1]['Team'],
    'Winner': x.iloc[0]['Team'] if x.iloc[0]['Map Won'] == 'Yes' else x.iloc[1]['Team'],
    'GameID': x.iloc[0]['GameID']
})).reset_index(drop=True)
h2h_wins = []
for team_a in team_stats['Team'].unique():
    for team_b in team_stats['Team'].unique():
        if team_a != team_b:
            matches = h2h_matches[((h2h_matches['Team_A'] == team_a) & (h2h_matches['Team_B'] == team_b)) | ((h2h_matches['Team_A'] == team_b) & (h2h_matches['Team_B'] == team_a))]
            if len(matches) >= 3:  # Require at least 3 matches
                win_rate = (matches['Winner'] == team_a).mean()
                h2h_wins.append({'Team_A': team_a, 'Team_B': team_b, 'H2H_Win_Rate': win_rate})
h2h_wins = pd.DataFrame(h2h_wins)

# Create match-level dataset
matches = team_stats.groupby('GameID').apply(lambda x: pd.Series({
    'Team_A': x.iloc[0]['Team'],
    'Team_B': x.iloc[1]['Team'],
    'Map': x.iloc[0]['Map'],
    'Winner': x.iloc[0]['Team'] if x.iloc[0]['Map Won'] == 'Yes' else x.iloc[1]['Team'],
    'GameID': x.iloc[0]['GameID'],
    'Team_A_Rating': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_Rating'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_Rating': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_Rating'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_Kills': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_Kills'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_Kills': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_Kills'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_KAST': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_KAST'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_KAST': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_KAST'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_ADR': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_ADR'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_ADR': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_ADR'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_Score': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_Score'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_Score': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_Score'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_First_Kills': rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]['Rolling_First_Kills'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[0]['Team']) & (rolling_stats['GameID'] < x.iloc[0]['GameID'])]) > 0 else 0,
    'Team_B_First_Kills': rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]['Rolling_First_Kills'].iloc[-1] if len(rolling_stats[(rolling_stats['Team'] == x.iloc[1]['Team']) & (rolling_stats['GameID'] < x.iloc[1]['GameID'])]) > 0 else 0,
    'Team_A_Map_Win_Rate': map_wins[(map_wins['Team'] == x.iloc[0]['Team']) & (map_wins['Map'] == x.iloc[0]['Map'])]['Map_Win_Rate'].iloc[0] if len(map_wins[(map_wins['Team'] == x.iloc[0]['Team']) & (map_wins['Map'] == x.iloc[0]['Map'])]) > 0 else 0.5,
    'Team_B_Map_Win_Rate': map_wins[(map_wins['Team'] == x.iloc[1]['Team']) & (map_wins['Map'] == x.iloc[1]['Map'])]['Map_Win_Rate'].iloc[0] if len(map_wins[(map_wins['Team'] == x.iloc[1]['Team']) & (map_wins['Map'] == x.iloc[1]['Map'])]) > 0 else 0.5,
    'Team_A_H2H_Win_Rate': h2h_wins[(h2h_wins['Team_A'] == x.iloc[0]['Team']) & (h2h_wins['Team_B'] == x.iloc[1]['Team'])]['H2H_Win_Rate'].iloc[0] if len(h2h_wins[(h2h_wins['Team_A'] == x.iloc[0]['Team']) & (h2h_wins['Team_B'] == x.iloc[1]['Team'])]) > 0 else 0.5,
    'Team_A_Tier': team_tiers[team_tiers['Team'] == x.iloc[0]['Team']]['Team_Tier'].iloc[0] if x.iloc[0]['Team'] in team_tiers['Team'].values else 1,
    'Team_B_Tier': team_tiers[team_tiers['Team'] == x.iloc[1]['Team']]['Team_Tier'].iloc[0] if x.iloc[1]['Team'] in team_tiers['Team'].values else 1
})).reset_index(drop=True)

# Create differential features
matches['Rating_Diff'] = matches['Team_A_Rating'] - matches['Team_B_Rating']
matches['Kills_Diff'] = matches['Team_A_Kills'] - matches['Team_B_Kills']
matches['KAST_Diff'] = matches['Team_A_KAST'] - matches['Team_B_KAST']
matches['ADR_Diff'] = matches['Team_A_ADR'] - matches['Team_B_ADR']
matches['Score_Diff'] = matches['Team_A_Score'] - matches['Team_B_Score']
matches['First_Kills_Diff'] = matches['Team_A_First_Kills'] - matches['Team_B_First_Kills']
matches['Map_Win_Rate_Diff'] = matches['Team_A_Map_Win_Rate'] - matches['Team_B_Map_Win_Rate']
matches['Tier_Diff'] = matches['Team_A_Tier'] - matches['Team_B_Tier']

# Cap outliers
diff_features = ['Rating_Diff', 'Kills_Diff', 'KAST_Diff', 'ADR_Diff', 'Score_Diff', 'First_Kills_Diff', 'Map_Win_Rate_Diff']
for feat in diff_features:
    matches[feat] = matches[feat].clip(lower=matches[feat].quantile(0.01), upper=matches[feat].quantile(0.99))

# Encode winner
matches['Target'] = matches.apply(lambda x: 1 if x['Winner'] == x['Team_A'] else 0, axis=1)

# Encode map
le_map = LabelEncoder()
matches['Map_Encoded'] = le_map.fit_transform(matches['Map'])

# Define training features
training_features = ['Rating_Diff', 'Kills_Diff', 'KAST_Diff', 'ADR_Diff', 'Score_Diff', 'First_Kills_Diff', 'Map_Win_Rate_Diff', 'Team_A_H2H_Win_Rate', 'Tier_Diff', 'Map_Encoded']
X = matches[training_features]
y = matches['Target']

# Temporal train-test split
game_ids = matches['GameID'].unique()
game_ids.sort()
split_idx = int(0.8 * len(game_ids))
train_ids = game_ids[:split_idx]
test_ids = game_ids[split_idx:]
train = matches[matches['GameID'].isin(train_ids)]
test = matches[matches['GameID'].isin(test_ids)]

X_train = train[training_features]
y_train = train['Target']
X_test = test[training_features]
y_test = test['Target']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Training with XGBoost

- Train an XGBoost classifier with isotonic calibration
- Perform hyperparameter tuning
- Evaluate on test set

In [7]:
# Initialize XGBoost classifier
xgb = XGBClassifier(random_state=42, eval_metric='logloss')

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Perform grid search
grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Calibrate the best model
best_model = grid_search.best_estimator_
calibrated_model = CalibratedClassifierCV(best_model, cv=10, method='isotonic')
calibrated_model.fit(X_train_scaled, y_train)

print('Best Parameters:', grid_search.best_params_)

# Predict on test set
y_pred = calibrated_model.predict(X_test_scaled)
y_pred_proba = calibrated_model.predict_proba(X_test_scaled)

# Evaluate
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))
print('Log Loss:', log_loss(y_test, y_pred_proba))

# Feature importance
feature_importance = pd.DataFrame({'Feature': training_features, 'Importance': best_model.feature_importances_})
print('Feature Importance:\n', feature_importance.sort_values(by='Importance', ascending=False))

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Accuracy: 0.8170731707317073
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.83      0.82        41
           1       0.82      0.80      0.81        41

    accuracy                           0.82        82
   macro avg       0.82      0.82      0.82        82
weighted avg       0.82      0.82      0.82        82

Log Loss: 0.4049059072073191
Feature Importance:
                Feature  Importance
6    Map_Win_Rate_Diff    0.525233
0          Rating_Diff    0.174107
3             ADR_Diff    0.074753
2            KAST_Diff    0.053343
1           Kills_Diff    0.046169
4           Score_Diff    0.037482
5     First_Kills_Diff    0.033781
8            Tier_Diff    0.031052
9          Map_Encoded    0.016616
7  Team_A_H2H_Win_Rate    0.007465


## Playoff Simulation

- Simulate a single-elimination playoff bracket for BLAST Austin Major
- Use BO3 format with random map selection
- Compute confidence intervals for probabilities
- Validate team data availability

In [8]:
# Define playoff teams
playoff_teams = ['FURIA', 'paiN', 'FaZe', 'The MongolZ', 'Natus Vincere', 'Vitality', 'Spirit', 'MOUZ']
map_pool = ['Mirage', 'Dust2', 'Ancient', 'Nuke', 'Vertigo']
print('Playoff Teams:', playoff_teams)

# Validate team data
print('\nTeam Data Summary:')
for team in playoff_teams:
    matches_played = len(rolling_stats[rolling_stats['Team'] == team])
    avg_rating = rolling_stats[rolling_stats['Team'] == team]['Rolling_Rating'].mean()
    tier = team_tiers[team_tiers['Team'] == team]['Team_Tier'].iloc[0] if team in team_tiers['Team'].values else 1
    print(f'{team}: {matches_played} matches, Avg Rolling Rating: {avg_rating:.2f}, Tier: {tier}')

# Function to simulate a single map
def simulate_match_single_map(team_a, team_b, map_name, rolling_stats, map_wins, h2h_wins, team_tiers):
    raw_features = ['Rolling_Rating', 'Rolling_Kills', 'Rolling_KAST', 'Rolling_ADR', 'Rolling_Score', 'Rolling_First_Kills']
    team_a_stats = rolling_stats[rolling_stats['Team'] == team_a][raw_features].iloc[-1:].mean().values
    team_b_stats = rolling_stats[rolling_stats['Team'] == team_b][raw_features].iloc[-1:].mean().values
    
    if np.any(np.isnan(team_a_stats)) or np.any(np.isnan(team_b_stats)):
        print(f"Warning: No stats for {team_a} or {team_b} on {map_name}. Using defaults.")
        team_a_stats = np.nan_to_num(team_a_stats, nan=0.0)
        team_b_stats = np.nan_to_num(team_b_stats, nan=0.0)
    
    diff_features = team_a_stats - team_b_stats
    team_a_map_win_rate = map_wins[(map_wins['Team'] == team_a) & (map_wins['Map'] == map_name)]['Map_Win_Rate'].iloc[0] if len(map_wins[(map_wins['Team'] == team_a) & (map_wins['Map'] == map_name)]) > 0 else 0.5
    team_b_map_win_rate = map_wins[(map_wins['Team'] == team_b) & (map_wins['Map'] == map_name)]['Map_Win_Rate'].iloc[0] if len(map_wins[(map_wins['Team'] == team_b) & (map_wins['Map'] == map_name)]) > 0 else 0.5
    map_win_rate_diff = team_a_map_win_rate - team_b_map_win_rate
    h2h_win_rate = h2h_wins[(h2h_wins['Team_A'] == team_a) & (h2h_wins['Team_B'] == team_b)]['H2H_Win_Rate'].iloc[0] if len(h2h_wins[(h2h_wins['Team_A'] == team_a) & (h2h_wins['Team_B'] == team_b)]) > 0 else 0.5
    team_a_tier = team_tiers[team_tiers['Team'] == team_a]['Team_Tier'].iloc[0] if team_a in team_tiers['Team'].values else 1
    team_b_tier = team_tiers[team_tiers['Team'] == team_b]['Team_Tier'].iloc[0] if team_b in team_tiers['Team'].values else 1
    tier_diff = team_a_tier - team_b_tier
    
    try:
        map_encoded = le_map.transform([map_name])[0]
    except ValueError:
        map_encoded = le_map.transform(['Mirage'])[0]
    
    match_features = np.append(np.append(np.append(diff_features, map_win_rate_diff), [h2h_win_rate, tier_diff]), map_encoded).reshape(1, -1)
    match_features_scaled = scaler.transform(match_features)
    prob = calibrated_model.predict_proba(match_features_scaled)[0][1]
    return prob

# Function to simulate a BO3 match
def simulate_bo3_match(team_a, team_b, map_pool):
    maps = np.random.choice(map_pool, size=3, replace=False)
    team_a_wins = 0
    probs = []
    for map_name in maps:
        prob = simulate_match_single_map(team_a, team_b, map_name, rolling_stats, map_wins, h2h_wins, team_tiers)
        probs.append(prob)
        if prob >= 0.5:
            team_a_wins += 1
        if team_a_wins == 2 or (len(maps) - team_a_wins) == 2:
            break
    avg_prob = np.mean(probs)
    winner = team_a if team_a_wins >= 2 else team_b
    
    # Bootstrap confidence interval
    bootstrap_probs = [np.mean(np.random.choice(probs, size=len(probs), replace=True)) for _ in range(100)]
    ci_lower, ci_upper = np.percentile(bootstrap_probs, [2.5, 97.5])
    return winner, avg_prob, (ci_lower, ci_upper)

# Simulate playoff bracket
def simulate_playoffs(teams):
    print('Quarterfinals:')
    winners = []
    for i in range(0, len(teams), 2):
        winner, prob, ci = simulate_bo3_match(teams[i], teams[i+1], map_pool)
        print(f'{teams[i]} vs {teams[i+1]} -> {winner} (Prob: {prob:.2f}, 95% CI: [{ci[0]:.2f}, {ci[1]:.2f}])')
        winners.append(winner)
    
    print('\nSemifinals:')
    semi_winners = []
    for i in range(0, len(winners), 2):
        winner, prob, ci = simulate_bo3_match(winners[i], winners[i+1], map_pool)
        print(f'{winners[i]} vs {winners[i+1]} -> {winner} (Prob: {prob:.2f}, 95% CI: [{ci[0]:.2f}, {ci[1]:.2f}])')
        semi_winners.append(winner)
    
    print('\nFinal:')
    champion, prob, ci = simulate_bo3_match(semi_winners[0], semi_winners[1], map_pool)
    print(f'{semi_winners[0]} vs {semi_winners[1]} -> {champion} (Prob: {prob:.2f}, 95% CI: [{ci[0]:.2f}, {ci[1]:.2f}])')
    return champion

# Run simulation
champion = simulate_playoffs(playoff_teams)
print(f'\nPredicted Tournament Champion: {champion}')

Playoff Teams: ['FURIA', 'paiN', 'FaZe', 'The MongolZ', 'Natus Vincere', 'Vitality', 'Spirit', 'MOUZ']

Team Data Summary:
FURIA: 19 matches, Avg Rolling Rating: 0.29, Tier: 3
paiN: 16 matches, Avg Rolling Rating: -0.53, Tier: 2
FaZe: 25 matches, Avg Rolling Rating: 0.37, Tier: 3
The MongolZ: 27 matches, Avg Rolling Rating: -0.13, Tier: 2
Natus Vincere: 14 matches, Avg Rolling Rating: 0.18, Tier: 3
Vitality: 20 matches, Avg Rolling Rating: 1.37, Tier: 3
Spirit: 17 matches, Avg Rolling Rating: 1.25, Tier: 3
MOUZ: 21 matches, Avg Rolling Rating: -0.18, Tier: 3
Quarterfinals:
FURIA vs paiN -> paiN (Prob: 0.98, 95% CI: [0.98, 0.98])
FaZe vs The MongolZ -> The MongolZ (Prob: 0.66, 95% CI: [0.35, 0.96])
Natus Vincere vs Vitality -> Vitality (Prob: 0.24, 95% CI: [0.06, 0.34])
Spirit vs MOUZ -> MOUZ (Prob: 0.65, 95% CI: [0.32, 0.97])

Semifinals:
paiN vs The MongolZ -> The MongolZ (Prob: 0.46, 95% CI: [0.01, 0.90])
Vitality vs MOUZ -> MOUZ (Prob: 1.00, 95% CI: [1.00, 1.00])

Final:
The MongolZ